In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# 1. Understand the dataset

# 1.1 Import the dataset

In [ ]:
service_requests = pd.read_csv("311_Service_Requests_from_2010_to_Present.csv", index_col = 0)

# 1.2 Visualize the Dataset

In [ ]:
service_requests.head()

In [ ]:
service_requests.tail()

# 1.3 Print the columns of the Dataframe

In [ ]:
service_requests.columns

# 1.4 Identify the shape of the Dataset

In [ ]:
service_requests.shape

# 1.5 Identify the variables with null values

In [ ]:
service_requests.isna().sum()

In [ ]:
percent_missing = service_requests.isnull().sum() * 100 / len(service_requests) 
missing_value_df = pd.DataFrame({'column_name': service_requests.columns, 'percent_missing': percent_missing}) 
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False) 
missing_value_df['percent_missing']= missing_value_df['percent_missing'].round(1) 
missing_value_df.reset_index(drop=True).head(20)

In [ ]:
missing_value_df['drop_flag'] = missing_value_df['percent_missing'].apply(lambda x: '1' if x >80 else '0') 
missing_value_df.reset_index(drop=True).head()

In [ ]:
service_requests.isna()

# 2. Perform basic data exploratory analysis

# 2.1 Draw a frequency plot to show the number of null values in each column of the DataFrame

In [ ]:
service_requests.isna().sum().plot(figsize=(10,10) ,kind='bar')

# 2.2 Missing Value Treatment

# 2.2.1 Remove the records whose Closed Date values are null

In [ ]:
service_requests['Closed Date'].isna().sum()

In [ ]:
service_requests.dropna(subset=['Closed Date'], inplace = True)

In [ ]:
service_requests['Closed Date'].isna().sum()

# 2.3 Analyze the date column, and remove entries that have an incorrect timeline

In [ ]:
service_requests.tail()

In [ ]:
service_requests['Created Date'] = pd.to_datetime(service_requests['Created Date'])
service_requests['Closed Date'] = pd.to_datetime(service_requests['Closed Date'])

In [ ]:
service_requests['Closed Date'].max()

In [ ]:
service_requests['Closed Date'].min()

# 2.3.1 Calculate the time elapsed in closed and creation date

In [ ]:
service_requests['Elapsed_Time'] = service_requests['Closed Date'] - service_requests['Created Date']
service_requests.head()

# 2.3.2 Convert the calculated date to seconds to get a better representation

In [ ]:
service_requests['Elapsed_Time'] = service_requests['Elapsed_Time'].dt.seconds
service_requests.head()

# 2.3.3 View the descriptive statistics for the newly created column

In [ ]:
service_requests['Elapsed_Time'].describe()

# 2.3.4 Check the number of null values in the Complaint_Type and City columns

In [ ]:
service_requests['Complaint Type'].isna().sum()

In [ ]:
service_requests['City'].isna().sum()

# 2.3.5 Impute the NA value with Unknown City

In [ ]:
service_requests['City'].replace(to_replace = np.nan, value = 'Unknown City', inplace=True)
service_requests.isna().sum()

# 2.3.6 Draw a frequency plot for the complaints in each city

In [ ]:
service_requests['City'].unique()

In [ ]:
service_requests['City'].value_counts().plot(kind = 'bar', figsize=(20,10))

# 2.3.7 Create a scatter and hexbin plot of the concentration of complaints across Brooklyn

In [ ]:
sr2 = service_requests[service_requests['City'] == "BROOKLYN"]
sr2['City'].value_counts()

In [ ]:
count = sr2['Complaint Type'].value_counts().reset_index()
count

In [ ]:
plt.figure(figsize=(20,10), dpi=80)
plt.scatter(x=sr2['X Coordinate (State Plane)'], y=sr2['Y Coordinate (State Plane)'])
plt.show()

In [ ]:
x = sr2['Longitude'].to_numpy()
y = sr2['Latitude'].to_numpy()

plt.hexbin(x, y, gridsize=(20,20), cmap=plt.cm.Purples_r)
plt.colorbar()
plt.show()

# 3. Find major types of complaints:

# 3.1 Plot a bar graph to show the types of complaints

In [ ]:
service_requests['Complaint Type'].value_counts().plot(kind='bar')

# 3.2 Check the frequency of various types of complaints for New York City

In [ ]:
sr_ny = service_requests[service_requests['City'] == 'NEW YORK']
sr_ny['Complaint Type'].value_counts()

# 3.3 Find the top 10 complaint types

In [ ]:
sr_ny = sr_ny['Complaint Type'].value_counts().reset_index()
sr_ny.sort_values(by='Complaint Type', ascending=False).head(10)

# 3.4 Display the various types of complaints in each city

In [ ]:
#First Way
gr = service_requests.groupby(by='City')
gr['Complaint Type'].unique()

In [ ]:
#Second Way
gr_cross = pd.crosstab(index= service_requests['City'], columns = service_requests['Complaint Type'])
gr_cross

# 3.5 Create a DataFrame, df_new, which contains cities as columns and complaint types in rows

In [ ]:
#I understood the question as the columns being different City Names and the rows being different Complaint Types
#If the other way around is meant, the answer would be:
#df_new = pd.crosstab(index= service_requests['City'], columns = service_requests['Complaint_Type'])
df_new = pd.crosstab(index= service_requests['Complaint Type'], columns = service_requests['City'])
df_new.head(10)

# 4. Visualize the major types of complaints in each city

# 4.1 Draw another chart that shows the types of complaints in each city in a single chart, where different colors show the different types of complaints

In [ ]:
cr = pd.crosstab(index= service_requests['City'], columns = service_requests['Complaint Type'])
com_plot = cr.plot(kind ='barh', figsize=(15,10), legend=True, stacked=True)
com_plot.legend(title = 'Complaints')

# 4.2 Sort the complaint types based on the average Request_Closing_Time grouping them for different locations

In [ ]:
service_requests['Request_Closing_Time'] = service_requests['Closed Date'] - service_requests['Created Date']

In [ ]:
service_requests.sort_values(by=['Request_Closing_Time'])
service_requests.groupby('City')['Request_Closing_Time'].mean()

# 5. See whether the average response time across different complaint types is similar (overall)

# 5.1 Visualize the average of Request_Closing_Time

In [ ]:
service_requests['Response_Time'] = (service_requests['Closed Date'] - service_requests['Created Date']).astype('timedelta64[h]')
service_requests.groupby('Complaint Type')['Response_Time'].mean().plot(kind='barh', rot=45, fontsize=10, figsize=(18, 6))

# 6. Identify the significant variables by performing statistical analysis using p-values

In [ ]:
mean= service_requests['Response_Time'].mean()
service_requests['Above_Average_Response_Time'] = service_requests.apply(lambda x: '1' if x['Response_Time'] > mean else '0', axis=1)
pd.crosstab(service_requests['Complaint Type'], service_requests['Above_Average_Response_Time'])

# 7. Perform a Kruskal-Wallis H test

In [ ]:
from scipy import stats

bronx = service_requests[service_requests['City']== 'BRONX'] 
x=bronx['Response_Time'].to_list() 
brooklyn= service_requests[service_requests['City']== 'BROOKLYN'] 
y=brooklyn['Response_Time'].to_list() 

#H0 -> All sample distributions are equal
#alpha for this case will be 0.05

stats.kruskal(x, y)

In [ ]:
#reject the Null Hypothesis, because the pvalue is less than alpha 0.5

# 8. Present your observations

In [ ]:
#High numbers of NA values for multiple columns in the dataset

#NYC, Brooklyn and Bronx have the highest number of Complaints

#w

#This differs for each city as seen in the data for NYC
#This could be due to the nature of each city(e.g: NYC is busier, whereas other cities are more suburban)
#This would need to be tested with further Hypothesis testing

#All the request closing times are relatively similar except animal in park which has a much higher value than
#the others